In [1]:
import ipyleaflet
import ipywidgets
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import datetime

In [2]:
path_inventario = '../LHC_Hidroweb/invent2.csv'

In [3]:
df = pd.read_csv(path_inventario, engine='python', sep='\t', delimiter=';', parse_dates=['UltimaAtualizacao'])

In [4]:
df.columns

Index(['RegistroID', 'Importado', 'Temporario', 'Removido',
       'ImportadoRepetido', 'BaciaCodigo', 'SubBaciaCodigo', 'RioCodigo',
       'EstadoCodigo', 'MunicipioCodigo', 'ResponsavelCodigo',
       'ResponsavelUnidade', 'ResponsavelJurisdicao', 'OperadoraCodigo',
       'OperadoraUnidade', 'OperadoraSubUnidade', 'TipoEstacao', 'Codigo',
       'Nome', 'CodigoAdicional', 'Latitude', 'Longitude', 'Altitude',
       'AreaDrenagem', 'TipoEstacaoEscala', 'TipoEstacaoRegistradorNivel',
       'TipoEstacaoDescLiquida', 'TipoEstacaoSedimentos',
       'TipoEstacaoQualAgua', 'TipoEstacaoPluviometro',
       'TipoEstacaoRegistradorChuva', 'TipoEstacaoTanqueEvapo',
       'TipoEstacaoClimatologica', 'TipoEstacaoPiezometria',
       'TipoEstacaoTelemetrica', 'PeriodoEscalaInicio', 'PeriodoEscalaFim',
       'PeriodoRegistradorNivelInicio', 'PeriodoRegistradorNivelFim',
       'PeriodoDescLiquidaInicio', 'PeriodoDescLiquidaFim',
       'PeriodoSedimentosInicio', 'PeriodoSedimentosFim',
      

In [5]:
df[['Latitude', 'Longitude']] = df[['Latitude', 'Longitude']].apply(lambda x: x.str.replace(',','.'))
df['Latitude'] = df['Latitude'].astype('float')
df['Longitude'] = df['Longitude'].astype('float')

In [6]:
df[['Latitude','Longitude']].head()

,Latitude,Longitude
0,-13.6500,-73.0833
1,-13.0167,-73.9833
2,-13.6500,-73.3667
3,-13.4333,-73.8333
4,-13.6500,-73.9500


In [7]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude,df.Latitude))

In [8]:
m01 = ipyleaflet.Map(zoom=2)

# Layers
# ## All points for HEATMAP
heatmap_all = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],
                                 radius=30, name='All point Heatmap')
m01.add_layer(heatmap_all)

radius_slider = ipywidgets.IntSlider(description='Radius', min=1, max=50, value=15)
ipywidgets.jslink((radius_slider,'value'),(heatmap_all,'radius'))
widget_control01 = ipyleaflet.WidgetControl(widget=radius_slider, position='bottomright')
m01.add_control(widget_control01)

## Last collection day
def date_location(_):
    heatmap_byLast.locations = [tuple(s) for s in df.loc[df['UltimaAtualizacao'] > date_slider.value, ['Latitude','Longitude']].to_numpy()]

heatmap_byLast = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],
                                    radius=30, name='By Date')
m01.add_layer(heatmap_byLast)

ipywidgets.jslink((radius_slider,'value'),(heatmap_byLast,'radius'))


date_slider = ipywidgets.SelectionSlider(options=pd.date_range(start='2000-01-01',end='2020-01-01', freq='M').to_numpy())
date_slider.observe(date_location,names='value')
widget_control02 = ipyleaflet.WidgetControl(widget=date_slider, position='topright')
m01.add_control(widget_control02)


# Controles
## Layer
control_layer = ipyleaflet.LayersControl(position='topright')
m01.add_control(control_layer)

## Fullscreen
control_fullscreen = ipyleaflet.FullScreenControl()
m01.add_control(control_fullscreen)

## Desenho
control_desenho = ipyleaflet.DrawControl()
feature_collection = {'type': 'FeatureCollection',
                      'features': []}
def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""   
    feature_collection['features'].append(geo_json)

control_desenho.on_draw(handle_draw)
m01.add_control(control_desenho)

## Mediçao
# control_medicao = ipyleaflet.MeasureControl(position='bottomleft',
#                                             active_color='orange',
#                                             primary_length_units='kilometers')
# m01.add_control(control_medicao)
# control_medicao.complete_color = 'red'

## Escala
control_escala = ipyleaflet.ScaleControl(position='bottomleft')
m01.add_control(control_escala)

m01

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [11]:
try:
    last_draw = feature_collection['features'][-1]['geometry']
    last_polygon = Polygon([(i[0], i[1]) for i in last_draw['coordinates'][0]])
    print(last_polygon)
except:
    print('Nenhum polígono desenhado')

POLYGON ((-55.975342 -11.523088, -33.475342 -24.527135, -30.311279 2.811371, -55.975342 -11.523088))


In [12]:
gdf.loc[(gdf['geometry'].within(last_polygon)),['Nome', 'Codigo']]

,Nome,Codigo
82,SÍTIO BICA,438038
84,BIXOPA,438074
85,TRUSSU,639050
86,LIMOEIRO DO NORTE,538009
87,FAZENDA MORADA NOVA,639075
...,...,...
36086,MATA DE SÃO JOÃO,1238030
36090,GUARATINGA,1639026
36091,CONCEIÇÃO DA BARRA,1839013
36093,PATOS,737039


In [13]:
gdf.loc[(gdf['geometry'].within(last_polygon)) & (gdf['UltimaAtualizacao']>date_slider.value) ,['Nome', 'Codigo']]

,Nome,Codigo
409,FAZENDA SÃO JOÃO,339045
415,FAZENDA ALEGRIA,442005
418,AÇUDE LAGOINHA,637010
422,PEDRA BRANCA,442019
423,PEDRINHAS,442020
...,...,...
35650,PCH SANTA EDWIGES III JUSANTE,1446015
35651,PCH SANTA EDWIGES III BARRAMENTO,21291500
35690,PCH LUIZ DIAS LINS JUSANTE,835209
35691,PCH LUIZ DIAS LINS BARRAMENTO,39480100
